# Actividad de cátedra 5: Batch Normalization
### Basado en la Tarea 1

Profesor de Cátedra: Pablo Estévez<br>
Profesor Auxiliar: Ignacio Reyes<br>
Ayudantes: Francisca Cona, Jhon Intriago, Pablo Montero, Óscar Pimentel, Esteban Reyes, Mauricio Romero

## Imports

In [2]:
import numpy as np
import tensorflow as tf

import itertools
import warnings
from sklearn.metrics import confusion_matrix as sk_conf_mat

# Se imprime la version de Tensorflow
print('Tensorflow version', tf.__version__)
# Se imprime si es que se esta utilizando GPU o no
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    warnings.warn('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.3.1


<ipython-input-2-c59910049910>:13: UserWarning: GPU device not found
  warnings.warn('GPU device not found')


## Preparación de la base de datos MNIST

In [3]:
BATCH_SIZE = 8

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

training_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(buffer_size=1024).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

## Definición de Clasificador MLP

In [18]:
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Flatten())
mlp.add(tf.keras.layers.BatchNormalization())
mlp.add(tf.keras.layers.Dense(300, activation="relu"))
mlp.add(tf.keras.layers.BatchNormalization())
mlp.add(tf.keras.layers.Dense(100, activation="relu"))
mlp.add(tf.keras.layers.BatchNormalization())
mlp.add(tf.keras.layers.Dense(10))

mlp.build(input_shape=(None, 28, 28))
mlp.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_8 (Dense)              (None, 10)               

## Entrenamiento de MLP

In [19]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
mlp.compile(optimizer, loss, metrics=['accuracy'])

In [20]:
mlp.fit(
    training_dataset, 
    epochs=10,
    steps_per_epoch=200,
    validation_data=test_dataset,
    validation_freq=1,
    callbacks=[tf.keras.callbacks.TensorBoard('./logs/yes_bn', update_freq=100)]
)

Epoch 1/10
200/200 [==============================] - 2s 12ms/step - loss: 1.0299 - accuracy: 0.6725 - val_loss: 0.6248 - val_accuracy: 0.8520
Epoch 2/10
200/200 [==============================] - 2s 9ms/step - loss: 0.5908 - accuracy: 0.8175 - val_loss: 0.3812 - val_accuracy: 0.8853
Epoch 3/10
200/200 [==============================] - 2s 8ms/step - loss: 0.5462 - accuracy: 0.8319 - val_loss: 0.3539 - val_accuracy: 0.8925
Epoch 4/10
200/200 [==============================] - 2s 9ms/step - loss: 0.4994 - accuracy: 0.8544 - val_loss: 0.3330 - val_accuracy: 0.9000
Epoch 5/10
200/200 [==============================] - 2s 8ms/step - loss: 0.4877 - accuracy: 0.8600 - val_loss: 0.3209 - val_accuracy: 0.9046
Epoch 6/10
200/200 [==============================] - 2s 8ms/step - loss: 0.5116 - accuracy: 0.8587 - val_loss: 0.3029 - val_accuracy: 0.9114
Epoch 7/10
200/200 [==============================] - 2s 9ms/step - loss: 0.3891 - accuracy: 0.8825 - val_loss: 0.3023 - val_accuracy: 0.9122
Epoch

In [16]:
%load_ext tensorboard
%tensorboard --logdir=.

# Instrucciones:
* Entrene la MLP sin batch normalization que se entrega en el código actual.
* Agregue 3 capas de batch normalization, cada una de ellas justo antes de cada capa fully-connected. Puede instanciar dicha capa con el comando tf.keras.layers.BatchNormalization()
* Modifique el directorio donde se guardan los logs de tensorboard para evitar escribir múltiples logs en la misma carpeta.
* Lance tensorboard y compare ambos entrenamientos.

# Preguntas
* ¿En qué caso se obtiene un mejor modelo?
* ¿Cómo cambian las curvas de aprendizaje?

# Entrega
* Entregue sus resultados y análisis en formato pdf.
* Inserte una captura de pantalla con las curvas de aprendizaje de tensorboard.
* Inserte una captura de pantalla con los grafos ambos modelos mostrados en tensorboard.

Nota: El "epoch" indicado en tensorboard corresponde a 200 iteraciones, no a una pasada completa por el dataset. Esto se configuró así para forzar al callback de keras a capturar métricas de validación de forma más seguida.
